In [ ]:
pip install yfinance

In [1]:
from helpers import *
import yfinance as yf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import norm
from datetime import date, timedelta, datetime

/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [2]:
from options_helpers import *
from scipy.ndimage import gaussian_filter1d
from scipy.interpolate import CubicSpline
from KalshiClientsBaseV2 import ExchangeClient

In [3]:
prod_email, prod_password = get_kalshi_creds()
prod_api_base = "https://trading-api.kalshi.com/trade-api/v2" 
exchange_client = ExchangeClient(exchange_api_base=prod_api_base, email = prod_email, password = prod_password)

In [5]:
annualized_rate = .05374
ir = np.log(1+annualized_rate)

In [4]:
ndx_df, spx_df = get_ndx_spx_data()
    
ndx_df = ndx_df[ndx_df['Date'] >= date(2024, 1, 1)][['Date', 'Open', 'Close']].reset_index(drop=True)
spx_df = spx_df[spx_df['Date'] >= date(2024, 1, 1)][['Date', 'Open', 'Close']].reset_index(drop=True) #get daily index data for 2024 onwards


In [ ]:
'''
gets hourly SPX and NDX options data on an hourly basis from some specified date onwards from Polygon and stores data in df
'''
import requests
params_ = {'adjusted': 'true', 'sort': 'asc', 'apiKey' : 'vGQU0FoV2y68f1aA4hFlncpgRNiJVroO'}
d = {'underlying': [], 'strike': [], 'type': [], 'date': [], 'ts': [], 'price': []}


exception_rate = []

# -- NDX below --

for idx, row in ndx_df.tail(1).iterrows():
    cur_dt = date.today()
    year = str(cur_dt.year - 2000)
    month = cur_dt.month
    if month < 10:
        month = '0' + str(month)
    else:
        month = str(month)
    day = cur_dt.day
    if day < 10:
        day = '0' + str(day)
    else:
        day = str(day)
        
    close_ = row['Open']
    #get OTM calls and puts data
    lb_call_strike = close_ + 10 - close_ % 10
    ub_put_strike = close_ - close_ % 10
    call_strikes = np.arange(lb_call_strike, lb_call_strike + 500, 10)
    put_strikes = np.arange(ub_put_strike - 500, ub_put_strike + 10, 10)
    strikes = list(call_strikes) + list(put_strikes)
    day_data_ret_err = 0
    for sp in strikes:

        put_ticker = 'NDXP' + year + month + day + 'P' +  str(int(sp)) + '000'
        call_ticker = 'NDXP'+ year + month + day + 'C' + str(int(sp)) + '000'
        put_url = 'https://api.polygon.io/v2/aggs/ticker/O:{}/range/{}/{}/{}/{}'.format(put_ticker, '1', 'hour', cur_dt.isoformat(), cur_dt.isoformat())
        call_url = 'https://api.polygon.io/v2/aggs/ticker/O:{}/range/{}/{}/{}/{}'.format(call_ticker, '1', 'hour', cur_dt.isoformat(), cur_dt.isoformat())
        
        r = requests.get(put_url, params=params_).json()
        try:
            for res in r['results']:
                option_close = res['o']
                ts = res['t']/1000
                d['strike'].append(int(sp))
                d['underlying'].append('NDX')
                d['type'].append('put')
                d['date'].append(cur_dt)
                d['ts'].append(datetime.fromtimestamp(ts))
                d['price'].append(option_close)
        except:
            day_data_ret_err += 1
            
        r = requests.get(call_url, params=params_).json()
        try:
            for res in r['results']:
                option_close = res['o']
                ts = res['t']/1000
                d['strike'].append(int(sp))
                d['underlying'].append('NDX')
                d['type'].append('call')
                d['date'].append(cur_dt)
                d['ts'].append(datetime.fromtimestamp(ts))
                d['price'].append(option_close)
        except:
            day_data_ret_err += 1
        
        
    if day_data_ret_err > 0:
        exception_rate.append(day_data_ret_err/(2*len(strikes)))

# -- SPX below --

for idx, row in spx_df.tail(1).iterrows():
    cur_dt = date.today()
    year = str(cur_dt.year - 2000)
    month = cur_dt.month
    if month < 10:
        month = '0' + str(month)
    else:
        month = str(month)
    day = cur_dt.day
    if day < 10:
        day = '0' + str(day)
    else:
        day = str(day)
        
    close_ = row['Open']
    #get OTM calls and puts data
    lb_call_strike = close_ + 5 - close_ % 5
    ub_put_strike = close_ - close_ % 5
    call_strikes = np.arange(lb_call_strike, lb_call_strike + 155, 5)
    put_strikes = np.arange(ub_put_strike - 150, ub_put_strike + 5, 5)
    strikes = list(call_strikes) + list(put_strikes)
    day_data_ret_err = 0
    for sp in strikes:

        put_ticker = 'SPXW' + year + month + day + 'P' + '0' + str(int(sp)) + '000'
        call_ticker = 'SPXW'+ year + month + day + 'C' + '0' + str(int(sp)) + '000'
        put_url = 'https://api.polygon.io/v2/aggs/ticker/O:{}/range/{}/{}/{}/{}'.format(put_ticker, '1', 'hour', cur_dt.isoformat(), cur_dt.isoformat())
        call_url = 'https://api.polygon.io/v2/aggs/ticker/O:{}/range/{}/{}/{}/{}'.format(call_ticker, '1', 'hour', cur_dt.isoformat(), cur_dt.isoformat())
        
        r = requests.get(put_url, params=params_).json()
        try:
            for res in r['results']:
                option_close = res['o']
                ts = res['t']/1000
                d['strike'].append(int(sp))
                d['underlying'].append('SPX')
                d['type'].append('put')
                d['date'].append(cur_dt)
                d['ts'].append(datetime.fromtimestamp(ts))
                d['price'].append(option_close)
        except:
            day_data_ret_err += 1
            
        r = requests.get(call_url, params=params_).json()
        try:
            for res in r['results']:
                option_close = res['o']
                ts = res['t']/1000
                d['strike'].append(int(sp))
                d['underlying'].append('SPX')
                d['type'].append('call')
                d['date'].append(cur_dt)
                d['ts'].append(datetime.fromtimestamp(ts))
                d['price'].append(option_close)
        except:
            day_data_ret_err += 1
        
        
    if day_data_ret_err > 0:
        exception_rate.append(day_data_ret_err/(2*len(strikes)))

print(sum(exception_rate)/len(exception_rate))
df = pd.DataFrame(data=d)

ndx_opt_data = df[df['underlying'] == 'NDX']
spx_opt_data = df[df['underlying'] == 'SPX']
ndx_df['date'] = ndx_df.apply(lambda x: x['Date'], axis=1)
spx_df['date'] = spx_df.apply(lambda x: x['Date'], axis=1)

ndx_df = ndx_df[['date', 'Open', 'Close']]
spx_df = spx_df[['date', 'Open', 'Close']]

ndx_opt_data = ndx_opt_data.merge(ndx_df, how='inner', on='date')

ndx_itm_data = ndx_opt_data[((ndx_opt_data['strike'] > ndx_opt_data['Open']) & (ndx_opt_data['type']=='put')) | ((ndx_opt_data['strike'] < ndx_opt_data['Open']) & (ndx_opt_data['type']=='call'))]
ndx_otm_data = ndx_opt_data[((ndx_opt_data['strike'] > ndx_opt_data['Open']) & (ndx_opt_data['type']=='call')) | ((ndx_opt_data['strike'] < ndx_opt_data['Open']) & (ndx_opt_data['type']=='put'))]

spx_opt_data = spx_opt_data.merge(spx_df, how='inner', on='date')
spx_itm_data = spx_opt_data[((spx_opt_data['strike'] > spx_opt_data['Open']) & (spx_opt_data['type']=='put')) | ((spx_opt_data['strike'] < spx_opt_data['Open']) & (spx_opt_data['type']=='call'))]
spx_otm_data = spx_opt_data[((spx_opt_data['strike'] > spx_opt_data['Open']) & (spx_opt_data['type']=='call')) | ((spx_opt_data['strike'] < spx_opt_data['Open']) & (spx_opt_data['type']=='put'))]


df_itm = pd.concat([ndx_itm_data, spx_itm_data])
df_otm = pd.concat([ndx_otm_data, spx_otm_data])
df_itm['hour'] = df_itm.apply(lambda x: x['ts'].hour, axis=1)
df_otm['hour'] = df_otm.apply(lambda x: x['ts'].hour, axis=1)

df_itm = df_itm[df_itm['hour']==9]
df_otm = df_otm[df_otm['hour']==9]

df_itm = df_itm.sort_values(by=['date', 'strike']).reset_index(drop=True)
df_otm = df_otm.sort_values(by=['date', 'strike']).reset_index(drop=True)


In [ ]:
df_otm[(df_otm['underlying']=='SPX') & (df_otm['strike'] > 5100) & (df_otm['strike'] < 5200)]

In [ ]:
df_otm[(df_otm['underlying']=='NDX') & (df_otm['strike'] > 17950) & (df_otm['strike'] < 18100)]

In [ ]:
df_otm['est_iv'] = df_otm.apply(lambda x: implied_volatility_call(x['price'], x['Open'], x['strike'], 1/252, ir) if x['type']=='call' else implied_volatility_put(x['price'], x['Open'], x['strike'], 1/252, ir) , axis=1)


c = S_0 N(d1) - K * e^(-rT) * N(d2)

c = e^ (-rT) * E [max(S-K, 0)]
 = e ^ (-rT) * sum (p(Z) * (Z-k)) for Z > k
 = e ^ (-rT) * p (S > k) * E[S-K| S > K]
 = e ^ (-rT) * p(S > K) * (E[S| S > K] - K)
 = e ^ (-rT) * p(S > K) * (E[S| S > K]) - K * e ^ (-rT) * p(S > K)
 
 so p(S>K) ~=~ N(d2)
 where d2 = (ln(S_0/K) + (r-(sigma**2)/2) * T)/(sigma * sqrt(T))
 

In [ ]:
def calc_geq_strike_prob(S_0, K, r, sigma, T):
    d2 = (np.log(S_0/K) + (r - (sigma**2)/2)*T)/(sigma * np.sqrt(T))
    return norm.cdf(d2)

In [ ]:
df_otm['geq_strike_prob'] = df_otm.apply(lambda x:calc_geq_strike_prob(x['Open'], x['strike'], ir, x['est_iv'], 1/252), axis=1)

In [ ]:
df_otm[(df_otm['underlying']=='SPX') & (df_otm['strike'] > 5000) & (df_otm['strike'] < 5250)]

In [ ]:
df_otm[(df_otm['underlying']=='NDX') & (df_otm['strike'] > 17900) & (df_otm['strike'] < 18100)]

In [6]:
#Kalshi SP500 above/below market goes back to Jan 19, 2024
lb = None

spx_d = {'bid': [], 'ask': [], 'lb': [], 'close': [], 'datetime': []}
for idx, row in spx_df.iterrows():
    dt = row['Date']
    if dt < date(2024, 1, 19):
        continue
    spx_close = float(row['Close'])
    event_ticker = 'INXU-' + str(dt.year-2000) + month_to_abbr[dt.month]
    if dt.day < 10:
        event_ticker += '0' + str(dt.day)
    else:
        event_ticker += str(dt.day)
    try:
        event = exchange_client.get_event(event_ticker)
    except:
        print(event_ticker)
        continue
    market_tickers = []
    ts = int(time.mktime(datetime(dt.year, dt.month, dt.day, 9, 0, 0).timetuple()))
    for m in event['markets']:
        ticker = m['ticker']
        subtitle = m['yes_sub_title']
        split = subtitle.split(' ')
        lb = float(split[0].replace(',',''))
        
        try:
            r = exchange_client.get_market_history(ticker, min_ts = ts)
        except HttpError:
            continue

        info = r['history'][0]
        first_ts = datetime.fromtimestamp(info['ts'])
        if first_ts.hour == 9:
            bb = info['yes_bid']
            ba = info['yes_ask']
            spx_d['bid'].append(bb)
            spx_d['ask'].append(ba)
            spx_d['lb'].append(lb)
            spx_d['datetime'].append(first_ts)
            spx_d['close'].append(spx_close)

spx_d = pd.DataFrame(data=spx_d)

In [8]:
spx_d[spx_d['datetime']<datetime(2024,1,20)]

,bid,ask,lb,close,datetime
0,99,100,4625.0,4839.810059,2024-01-19 09:44:52
1,99,100,4650.0,4839.810059,2024-01-19 09:44:49
2,99,100,4675.0,4839.810059,2024-01-19 09:44:46
3,99,100,4700.0,4839.810059,2024-01-19 09:44:41
4,91,100,4750.0,4839.810059,2024-01-19 09:00:07
5,83,90,4775.0,4839.810059,2024-01-19 09:00:17
6,52,56,4800.0,4839.810059,2024-01-19 09:00:17
7,18,100,4825.0,4839.810059,2024-01-19 09:00:17
8,3,7,4850.0,4839.810059,2024-01-19 09:00:17
9,0,1,4875.0,4839.810059,2024-01-19 09:45:00


In [9]:
from KalshiClientsBaseV2 import HttpError
lb = None

ndx_d = {'bid': [], 'ask': [], 'lb': [], 'close': [], 'datetime': []}
for idx, row in ndx_df.iterrows():
    dt = row['Date']
    if dt < date(2024, 1, 19):
        continue
    ndx_close = float(row['Close'])
    event_ticker = 'NASDAQ100U-' + str(dt.year-2000) + month_to_abbr[dt.month]
    if dt.day < 10:
        event_ticker += '0' + str(dt.day)
    else:
        event_ticker += str(dt.day)
    try:
        event = exchange_client.get_event(event_ticker)
    except:
        continue

    ts = int(time.mktime(datetime(dt.year, dt.month, dt.day, 9, 30, 0).timetuple()))
    for m in event['markets']:
        ticker = m['ticker']
        subtitle = m['yes_sub_title']
        split = subtitle.split(' ')
        lb = float(split[0].replace(',',''))
    
        try:
            r = exchange_client.get_market_history(ticker, min_ts = ts)
        except HttpError:
            continue

        info = r['history'][0]
        first_ts = datetime.fromtimestamp(info['ts'])
        if first_ts.hour == 9:
            bb = info['yes_bid']
            ba = info['yes_ask']
            ndx_d['bid'].append(bb)
            ndx_d['ask'].append(ba)
            ndx_d['lb'].append(lb)
            ndx_d['datetime'].append(first_ts)        
            ndx_d['close'].append(ndx_close)


ndx_d = pd.DataFrame(data=ndx_d)

In [10]:
ndx_d[ndx_d[]]

,bid,ask,lb,close,datetime
0,99,100,16600.0,17314.000000,2024-01-19 09:46:10
1,99,100,16700.0,17314.000000,2024-01-19 09:46:06
2,94,100,16800.0,17314.000000,2024-01-19 09:30:00
3,89,99,16900.0,17314.000000,2024-01-19 09:30:00
4,0,100,17000.0,17314.000000,2024-01-19 09:30:00
...,...,...,...,...,...
411,29,32,18000.0,18032.205078,2024-03-19 09:30:00
412,8,12,18100.0,18032.205078,2024-03-19 09:30:01
413,0,4,18200.0,18032.205078,2024-03-19 09:30:00
414,0,1,18300.0,18032.205078,2024-03-19 09:40:53
